In [1]:
import folium
import pandas as pd

In [1]:
import folium
show_map = folium.Map(location=[25.046273,121.517498],zoom_start=13,tiles = 'cartodbpositron')
show_map
folium.Circle(location=[25.046273,121.517498],
                       popup='text',
                       radius=500,
                       color='crimson',
                       fill=True,
                       fill_color='crimson').add_to(show_map)
show_map

In [2]:
#捷運路線點
df = pd.read_json(r'C:\Users\n\01.MRT_file\TRTC.json',encoding='utf-8')
del df['LineNo']
del df['LineID']
del df['UpdateTime']
LineName = []
for i in range(len(df)):
    name = df['LineName'][i]['Zh_tw']
    LineName.append(name)
print(LineName)

['淡水信義線', '文湖線', '環狀線', '板南線', '松山新店線', '中和新蘆線']


# 1.淡水信義線 經緯度分割

In [3]:
#去除檔頭檔尾
string = (df['Geometry'][0])[17:-2]
#去除字串中間的括弧
remap = {ord('(') : None,ord(')') : None}
string1 = string.translate(remap)
#經緯度分開
line0 = string1.split(',')
#經緯度分開並放入 2個list
line0_lng = []
line0_lat = []
for i in range(len(line0)):
    line0_lng.append(float(line0[i].split(' ')[0]))
    line0_lat.append(float(line0[i].split(' ')[1]))

# 2.文湖線 經緯度分割

In [4]:
#去除檔頭檔尾
string = (df['Geometry'][1])[11:-1]
string
#去除字串中間的括弧
## remap = {ord('(') : None,ord(')') : None}
## string1 = string.translate(remap)
#經緯度分開
line0 = string.split(',')
#經緯度分開並放入 2個list
line1_lng = []
line1_lat = []
for i in range(len(line0)):
    line1_lng.append(float(line0[i].split(' ')[0]))
    line1_lat.append(float(line0[i].split(' ')[1]))

# 3.環狀線 經緯度分割

In [5]:
#去除檔頭檔尾
string = (df['Geometry'][2])[11:-1]
string
#去除字串中間的括弧
## remap = {ord('(') : None,ord(')') : None}
## string1 = string.translate(remap)
#經緯度分開
line0 = string.split(',')
#經緯度分開並放入 2個list
line2_lng = []
line2_lat = []
for i in range(len(line0)):
    line2_lng.append(float(line0[i].split(' ')[0]))
    line2_lat.append(float(line0[i].split(' ')[1]))

# 4.板南線 經緯度分割

In [6]:
#去除檔頭檔尾
string = (df['Geometry'][3])[17:-2]
string
#去除字串中間的括弧
remap = {ord('(') : None,ord(')') : None}
string1 = string.translate(remap)
#經緯度分開
line0 = string1.split(',')
#經緯度分開並放入 2個list
line3_lng = []
line3_lat = []
for i in range(len(line0)):
    line3_lng.append(float(line0[i].split(' ')[0]))
    line3_lat.append(float(line0[i].split(' ')[1]))

# 5.松山新店線 經緯度分割

In [7]:
#去除檔頭檔尾
string = (df['Geometry'][4])[17:-2]
string
#去除字串中間的括弧
remap = {ord('(') : None,ord(')') : None}
string1 = string.translate(remap)
#經緯度分開
line0 = string1.split(',')
#經緯度分開並放入 2個list
line4_lng = []
line4_lat = []
for i in range(len(line0)):
    line4_lng.append(float(line0[i].split(' ')[0]))
    line4_lat.append(float(line0[i].split(' ')[1]))

# 6.中和新蘆線 經緯度分割

In [8]:
#去除檔頭檔尾
string = (df['Geometry'][5])[17:-2]
string
#去除字串中間的括弧
remap = {ord('(') : None,ord(')') : None}
string1 = string.translate(remap)
#經緯度分開
line0 = string1.split(',')
#經緯度分開並放入 2個list
line5_lng = []
line5_lat = []
for i in range(len(line0)):
    line5_lng.append(float(line0[i].split(' ')[0]))
    line5_lat.append(float(line0[i].split(' ')[1]))

# 選站名,時段'0-2',3-5','6-8','9-11','12-14','15-17','18-20','21-23'

In [9]:
# 1/(max(data['人數'])/630)

In [10]:
def choose_station_range(station,Trange):
    MRT_time_range = pd.read_csv(r'C:\Users\n\01.MRT_file\MRT_time_range.csv')
    MRT_time_range.rename(columns={'1900-01-01 00:00:00':'0~2',
                                   '1900-01-01 03:00:00':'3~5',
                                  '1900-01-01 06:00:00':'6~8',
                                  '1900-01-01 09:00:00':'9~11',
                                  '1900-01-01 12:00:00':'12~14',
                                  '1900-01-01 15:00:00':'15~17',
                                  '1900-01-01 18:00:00':'18~20',
                                  '1900-01-01 21:00:00':'21~23'},inplace=True)
    choose = MRT_time_range["進站"] == station
    MRT_time_range = MRT_time_range[choose]
    MRT_time_range = MRT_time_range[['進站','出站',Trange]]
    MRT_time_range = MRT_time_range.reset_index(drop = True)
    return MRT_time_range

In [11]:
def visualize(station):
    m = folium.Map(location=[25.046273,121.517498],zoom_start=11,tiles = 'cartodbpositron')
    #底圖形式 https://deparkes.co.uk/2016/06/10/folium-map-tiles/
    points00,points01,points10,points20,points30,points40,points41,points50,points51 = [],[],[],[],[],[],[],[],[]
    #weight=寬度
    #opacity=透明度
    #紅線 淡水信義線
    for a in range(334,len(line0_lat)):
        points00.append([line0_lat[a], line0_lng[a]])
    for aa in range(len(line0_lat)-250):
        points01.append([line0_lat[aa], line0_lng[aa]])
    folium.PolyLine(points00,weight=3,opacity=0.8, color='red').add_to(m)
    folium.PolyLine(points01,weight=3,opacity=0.8, color='red').add_to(m)
    #文湖線
    for i in range(len(line1_lat)):
        points10.append([line1_lat[i], line1_lng[i]])
    folium.PolyLine(points10,weight=3,opacity=0.8, color='brown').add_to(m)
    #環狀線
    for i in range(len(line2_lat)):
        points20.append([line2_lat[i], line2_lng[i]])
    folium.PolyLine(points20,weight=3,opacity=0.8, color='yellow').add_to(m)
    #板南線
    for i in range(len(line3_lat)):
        points30.append([line3_lat[i], line3_lng[i]])
    folium.PolyLine(points30,weight=3,opacity=0.8, color='blue').add_to(m)
    #松山新店線
    for i in range(754,len(line4_lat)):
        points40.append([line4_lat[i], line4_lng[i]])
    for ii in range(len(line4_lat)-328):
        points41.append([line4_lat[ii], line4_lng[ii]])     
    folium.PolyLine(points40,weight=3,opacity=0.8, color='green').add_to(m)
    folium.PolyLine(points41,weight=3,opacity=0.8, color='green').add_to(m)
    #中和新蘆線
    for i in range(256,len(line5_lat)):
        points50.append([line5_lat[i], line5_lng[i]])
    for ii in range(len(line5_lat)-285):
        points51.append([line5_lat[ii], line5_lng[ii]])
    folium.PolyLine(points50,weight=3,opacity=0.8, color='orange').add_to(m)
    folium.PolyLine(points51,weight=3,opacity=0.8, color='orange').add_to(m)
    #-------------------------虛線以上畫底圖-------------------------------------------------------
    #讀檔(捷運出口點位)
    location = pd.read_csv(r'C:\Users\n\01.MRT_file\MRT_location.csv',encoding='utf-8')
    location
    #篩選出每個站第一二個出入口經緯度
    fliter1 = (location['出入口編號'] == '0')
    fliter2 = (location['出入口編號'] == '1')
    fliter3 = (location['出入口編號'] == 'M4')
    location = location[(fliter1 | fliter2 | fliter3)]
    #取出站點名子
    #將站點名子放入station_name
    station_name = []
    exit_name = list(location['出入口名稱'])
    for i in range(len(location)):
        name = ((exit_name[i]).split('站'))[0]
        station_name.append(name)
    #刪除無用columns
    del location['項次']
    del location['出入口名稱']
    del location['出入口編號']
    #改站名(要配合人流資料)
    for i in range(len(station_name)):
        if station_name[i] == '台北車':
            station_name[i] = '台北車站'
        if station_name[i] == '大橋頭':
            station_name[i] = '大橋頭站'
        if station_name[i] == '臺北橋':
            station_name[i] = '台北橋'
    location['出站'] = station_name
    data = pd.merge(station,location,on=['出站'])
    column = data.columns
    data.rename(columns={column[2]:'人數'},inplace=True)
    size = 1/(max(data['人數'])/630)#圓圈大小參數
    for i in range(0,len(data)):
        if data['出站'][i] == data['進站'][0]:
            folium.Marker(location=[data['緯度'][i],data['經度'][i]],popup=data['進站'][0],
                          icon=folium.Icon(icon='arrow-down',color='black')).add_to(m)
#             color='red',
        if data['經度'][i] == None:
            continue
        folium.Circle(location=[data.iloc[i]['緯度'], data.iloc[i]['經度']],
                       popup=[data.iloc[i]['出站'],data.iloc[i]['人數']],
                       radius=data.iloc[i]['人數']*size,
                       color='crimson',
                       fill=True,
                       fill_color='crimson').add_to(m)
    return m

In [12]:
import matplotlib.pyplot as plt # plt 用於顯示圖片
import matplotlib.image as mpimg # mpimg 用於讀取圖片
import numpy as np
lena = mpimg.imread(r'C:\Users\n\01.MRT_file\TaipeiMRT_station.jpg') 
plt.figure(figsize=(200, 100),dpi = 200)
# lena.shape #(512, 512, 3)
plt.imshow(lena) # 顯示圖片
plt.axis('off') # 不顯示座標軸
plt.show()

In [13]:
#選站名,時段'0~2',3~5','6~8','9~11','12~14','15~17','18~20','21~23'

In [14]:
station = choose_station_range('新店','15~17')
show = visualize(station)

In [15]:
show

In [16]:
#轉成網頁
import os
show.save(os.path.join(r'C:\Users\nick\01.MRT_file', 'MRT_visulize.html'))

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\nick\\01.MRT_file\\MRT_visulize.html'